# Analysing the supercell output of big-box PDF analysis of LaMnO$_3$

In [ ]:
import pymatgen as pymatgen
import numpy as np
import matplotlib.pyplot as plt
import pickle
import os
import pandas as pd

In [ ]:
%matplotlib qt

In [ ]:
import sys
sys.path.insert(1,r"..\code")
from van_vleck_calculator import *

## Load CIF

In [ ]:
from pymatgen.core.structure import Structure

if os.path.exists("LaMnO3-octahedra.pickle"):
    f = open("LaMnO3-octahedra.pickle","rb")
    LaMnO3 = pickle.load(f)
else:
    LaMnO3 = Structure.from_file("lmo_10x10x8.cif")
    LaMnO3.to("lmo_out.cif")
    f = open("LaMnO3-octahedra.pickle","wb")
    pickle.dump(LaMnO3,f)

## List Mn sites

In [ ]:
Mn_sites = [i for i in LaMnO3 if str(i.specie)=="Mn"]

## Load octahedra

In [ ]:
def LoadOctahedra(cif_data,elementName,Run=False):
    """
    If you want to force the code to re-calculate, set Run=True. Otherwise it will
    check to see if it has already done the calculation.
    """
    picklename = elementName + "_octahedra_pickle.txt"
    if Run or (os.path.exists(picklename) == False):  
        octahedra = []
  	
        for i in range(0,len(cif_data)): 
            if ( str(cif_data[i].specie) == elementName ):
                
                print(i)
                
                octahedron = Octahedron(
                    cif_data[i],
                    cif_data,
                    ligands_max_distance=3.5,
                    possible_ligands=["O"],
                )
                octahedra.append(octahedron)

        with open(picklename, 'wb') as f:
            pickle.dump(octahedra, f)
        return octahedra
    else:
        with open(picklename,'rb') as f:
            return pickle.load(f)

In [ ]:
run_here = False
MnO6_octahedra = LoadOctahedra(LaMnO3,"Mn",Run=run_here)

## prepare sites for octahedral_centre argument and vector for specified_axes (van Vleck)

In [ ]:
Mn_sites_singleCell = [
    [0,0.5,0], [0.5,0,0.5],
    [0,0.5,0.5], [0.5,0,0], 
]

In [ ]:
# Due to tilting, all Mn sites are calculated approximately along these axes
v1 = [[ 3 , 2 , -0.5], [-2.25 , 3.5 , 0.5], [ 2.75 , -0.375, 15.   ]]
v2 = [[2.25 , 3.5 , -0.5], [ -3 , 2 , 0.5], [ 2.75 , 0.375, 15.   ]]
v3 = [[ 3 , 2 , 0.5], [-2.25 , 3.5 , -0.5], [ -2.75 , 0.375, 15.   ]]
v4 = [[2.25 , 3.5 , 0.5], [ 3 , -2 , 0.5], [ -2.75 , -0.375, 15.   ]]

vectors = [v1,v2,v3,v4]

In [ ]:
def GenerateSites(CIF,poss_sites,a,b,c):
    """
    This function generates all the perfect sites for the Van Vleck calculation.
    Also generates an appropriate vector
    """
    
    sites = []
    vector_indices = []
    
    for i in range(0,a+1):
        for j in range(0,b+1):
            for k in range(0,c+1):
                for m in range(0,len(poss_sites)):
                    l = poss_sites[m]
                    sites.append(
                        [
                         (i+l[0])*CIF.lattice.a/a,
                         (j+l[1])*CIF.lattice.b/b,
                         (k+l[2])*CIF.lattice.c/c + (j+l[1])*CIF.lattice.b/b *np.tan(np.radians(90-CIF.lattice.alpha))
                        ]
                    )
                    vector_indices.append(m)
    
    return sites , vector_indices

In [ ]:
sites_Mn, vector_indices_Mn = GenerateSites(LaMnO3,Mn_sites_singleCell,10,10,8)

In [ ]:
# This function will give the ideal coordinates for a Ni atom based on the LPs of the riet cif
# sites should be GenerateNiSites, coords will be coordinates of this specific site
# struc will be the CIF of the supercell
# It will also return the index of the vector which will be used for specified / starting vectors
def perfectCoords(sites,vector_indices,octahedron,struc):
        
    dimensions_of_cell = [struc.lattice.a,struc.lattice.b,struc.lattice.c]
    
    coords = [
        octahedron.core_site.a * dimensions_of_cell[0], 
        octahedron.core_site.b * dimensions_of_cell[1], 
        octahedron.core_site.c * dimensions_of_cell[2] 
               + octahedron.core_site.b * dimensions_of_cell[1]*np.tan(np.radians(90-struc.lattice.alpha)) 
    ]
    
    viable_sites = []
    viable_site_vectors = []
    
    for i in range(0,len(sites)):
        site = sites[i]
        isViable = True
        for j in [0,1,2]:
            maxVal = max([k.coords[j] for k in octahedron.ligands])
            minVal = min([k.coords[j] for k in octahedron.ligands])
            if (site[j] > maxVal) or (site[j] < minVal):
                if True not in [(p.coords[j] >dimensions_of_cell[j]) or (p.coords[j] <0) for p in octahedron.ligands]: #this is to prevent periodicity problems
                    isViable = False
        if isViable:
            viable_sites.append(site)
            viable_site_vectors.append(vector_indices[i])
    
    if len(viable_sites) == 0:
        raise Exception("no viable sites")
        
    printDiff = False
                                        
    differences = []
    for i in viable_sites:
        diff = np.array([(i[j]-coords[j])**2 for j in [0,1,2]])
        differences.append(np.sum(diff))
    index = np.array(differences).argmin()
    vector = vectors[viable_site_vectors[index]]

    return viable_sites[index],vector

## Calculations for LaMnO$_3$ supercell

In [ ]:
failed_automated = []
failed_specified = []

In [ ]:
def RunCalculations(octahedra_data,Run,perfect_sites,possible_vectors,PDF_supercell):
    """
    If you want to force the code to re-calculate, set Run=True. Otherwise it will
    check to see if it has already done the calculation.
    """
    picklename = str(octahedra_data[0].core_site.specie) + "_" + "_calc_pickle.txt"
    if Run or (os.path.exists(picklename) == False):  
                
        VanVleck_Q3_1_automated = []
        VanVleck_Q3_2_automated = []
        VanVleck_Q3_3_automated = []
        VanVleck_Q1_automated = []
        VanVleck_Q2_automated = []
        VanVleck_Q4_automated = []
        VanVleck_Q5_automated = []
        VanVleck_Q6_automated = []
        VanVleck_arctan_automated = []
        VanVleck_mag_automated = []
        VanVleck_pairs_automated = []
        
        VanVleck_Q3_1_specified = []
        VanVleck_Q3_2_specified = []
        VanVleck_Q3_3_specified = []
        VanVleck_Q1_specified = []
        VanVleck_Q2_specified = []
        VanVleck_Q4_specified = []
        VanVleck_Q5_specified = []
        VanVleck_Q6_specified = []
        VanVleck_arctan_specified = []
        VanVleck_mag_specified = []
        VanVleck_pairs_specified = []
        
        VanVleck_Q3_1_ignore = []
        VanVleck_Q3_2_ignore = []
        VanVleck_Q3_3_ignore = []
        VanVleck_Q1_ignore = []
        VanVleck_Q2_ignore = []
        VanVleck_Q4_ignore = []
        VanVleck_Q5_ignore = []
        VanVleck_Q6_ignore = []
        VanVleck_arctan_ignore = []
        VanVleck_mag_ignore = []
        VanVleck_pairs_ignore = []
        
        bond_angle_variance = []
        
        total_number = len(octahedra_data)
        
        for i in range(0,total_number): 
            octahedron = octahedra_data[i]
            print("{} of {}: centred on {}".format(i,total_number,octahedron.core_site.coords))
                        
            crystal_coords , specified_axes = perfectCoords(
                perfect_sites,
                possible_vectors,
                octahedron,
                PDF_supercell
            )

            #automate axis finding
            
            print("\tAutomated with starting_axes {}".format(specified_axes))
            
            VanVleck_automated = octahedron.calculate_degenerate_Q3_van_vleck_modes(
                octahedral_centre=crystal_coords,
                specified_axes=specified_axes,
                output_pairs=False,
                omit_failed_rotations=True,
            )
            VanVleck_Q3_1_automated.append(VanVleck_automated[0])
            VanVleck_Q3_2_automated.append(VanVleck_automated[1])
            VanVleck_Q3_3_automated.append(VanVleck_automated[2])
            local_modes_automated , pairs_automated = octahedron.calculate_van_vleck_distortion_modes(
                octahedral_centre=crystal_coords,
                specified_axes=specified_axes,
                output_pairs=True,
                omit_failed_rotations=True
            )
            VanVleck_Q1_automated.append(local_modes_automated[0])
            VanVleck_Q2_automated.append(local_modes_automated[1])
            VanVleck_Q4_automated.append(local_modes_automated[3])
            VanVleck_Q5_automated.append(local_modes_automated[4])
            VanVleck_Q6_automated.append(local_modes_automated[5])
            VanVleck_angles_automated = octahedron.calculate_van_vleck_jahn_teller_params(
                octahedral_centre=crystal_coords,
                specified_axes=specified_axes,
                output_pairs=False,
                omit_failed_rotations=True
            )
            VanVleck_arctan_automated.append(VanVleck_angles_automated[1])
            VanVleck_mag_automated.append(VanVleck_angles_automated[0])
            VanVleck_pairs_automated.append(pairs_automated)

            #specify axes without automated rotation
            
            print("\tSpecified with specified_axes {}".format(specified_axes))
            
            VanVleck_specified = octahedron.calculate_degenerate_Q3_van_vleck_modes(
                octahedral_centre=crystal_coords,
                specified_axes=specified_axes,
                automatic_rotation=False
            )
            VanVleck_Q3_1_specified.append(VanVleck_specified[0])
            VanVleck_Q3_2_specified.append(VanVleck_specified[1])
            VanVleck_Q3_3_specified.append(VanVleck_specified[2])
            local_modes_specified , pairs_specified = octahedron.calculate_van_vleck_distortion_modes(
                octahedral_centre=crystal_coords,
                specified_axes=specified_axes,
                output_pairs=True,
                automatic_rotation=False
            )
            VanVleck_Q1_specified.append(local_modes_specified[0])
            VanVleck_Q2_specified.append(local_modes_specified[1])
            VanVleck_Q4_specified.append(local_modes_specified[3])
            VanVleck_Q5_specified.append(local_modes_specified[4])
            VanVleck_Q6_specified.append(local_modes_specified[5])
            VanVleck_angles_specified = octahedron.calculate_van_vleck_jahn_teller_params(
                octahedral_centre=crystal_coords,
                specified_axes=specified_axes,
                automatic_rotation=False
            )
            VanVleck_arctan_specified.append(VanVleck_angles_specified[1])
            VanVleck_mag_specified.append(VanVleck_angles_specified[0])
            VanVleck_pairs_specified.append(pairs_specified)
            
            #ignore angular distortion
            
            print("\tIgnore angular distortion. specified_axes are {}".format(specified_axes))
            
            VanVleck_ignore = octahedron.calculate_degenerate_Q3_van_vleck_modes(
                octahedral_centre=crystal_coords,
                ignore_angular_distortion=True,
                specified_axes=specified_axes,
            )
            VanVleck_Q3_1_ignore.append(VanVleck_ignore[0])
            VanVleck_Q3_2_ignore.append(VanVleck_ignore[1])
            VanVleck_Q3_3_ignore.append(VanVleck_ignore[2])
            local_modes_ignore  = octahedron.calculate_van_vleck_distortion_modes(
                octahedral_centre=crystal_coords,
                ignore_angular_distortion=True,
                specified_axes=specified_axes,
            )
            VanVleck_Q1_ignore.append(local_modes_ignore[0])
            VanVleck_Q2_ignore.append(local_modes_ignore[1])
            VanVleck_Q4_ignore.append(local_modes_ignore[3])
            VanVleck_Q5_ignore.append(local_modes_ignore[4])
            VanVleck_Q6_ignore.append(local_modes_ignore[5])
            VanVleck_angles_ignore = octahedron.calculate_van_vleck_jahn_teller_params(
                octahedral_centre=crystal_coords,
                ignore_angular_distortion=True,
                specified_axes=specified_axes,
            )
            VanVleck_arctan_ignore.append(VanVleck_angles_ignore[1])
            VanVleck_mag_ignore.append(VanVleck_angles_ignore[0])
            
            #BAV
            bond_angle_variance.append(octahedron.calculate_bond_angle_variance())

        print("done")
        
        d = {'VanVleck_Q3_1_automated':VanVleck_Q3_1_automated,
             'VanVleck_Q3_2_automated':VanVleck_Q3_2_automated,
             'VanVleck_Q3_3_automated':VanVleck_Q3_3_automated,
             'VanVleck_Q1_automated': VanVleck_Q1_automated,
             'VanVleck_Q2_automated': VanVleck_Q2_automated,
             'VanVleck_Q4_automated': VanVleck_Q4_automated,
             'VanVleck_Q5_automated': VanVleck_Q5_automated,
             'VanVleck_Q6_automated': VanVleck_Q6_automated,
             'VanVleck_arctan_automated': VanVleck_arctan_automated,
             'VanVleck_mag_automated': VanVleck_mag_automated,
             'VanVleck_pairs_automated': VanVleck_pairs_automated,
             
            'VanVleck_Q3_1_specified':VanVleck_Q3_1_specified,
            'VanVleck_Q3_2_specified':VanVleck_Q3_2_specified,
            'VanVleck_Q3_3_specified':VanVleck_Q3_3_specified,
            'VanVleck_Q1_specified': VanVleck_Q1_specified,
            'VanVleck_Q2_specified': VanVleck_Q2_specified,
            'VanVleck_Q4_specified': VanVleck_Q4_specified,
            'VanVleck_Q5_specified': VanVleck_Q5_specified,
            'VanVleck_Q6_specified': VanVleck_Q6_specified,
            'VanVleck_arctan_specified': VanVleck_arctan_specified,
            'VanVleck_mag_specified': VanVleck_mag_specified,
            'VanVleck_pairs_specified': VanVleck_pairs_specified,
             
            'VanVleck_Q3_1_ignore':VanVleck_Q3_1_ignore,
            'VanVleck_Q3_2_ignore':VanVleck_Q3_2_ignore,
            'VanVleck_Q3_3_ignore':VanVleck_Q3_3_ignore,
            'VanVleck_Q1_ignore': VanVleck_Q1_ignore,
            'VanVleck_Q2_ignore': VanVleck_Q2_ignore,
            'VanVleck_Q4_ignore': VanVleck_Q4_ignore,
            'VanVleck_Q5_ignore': VanVleck_Q5_ignore,
            'VanVleck_Q6_ignore': VanVleck_Q6_ignore,
            'VanVleck_arctan_ignore': VanVleck_arctan_ignore,
            'VanVleck_mag_ignore': VanVleck_mag_ignore,
             
            'bond_angle_variance': bond_angle_variance,
            }
        
        with open(picklename, 'wb') as f:
            pickle.dump(d, f)
        return d
    else:
        with open(picklename,'rb') as f:
            return pickle.load(f)


In [ ]:
run_here = False

data_dictionary = RunCalculations(MnO6_octahedra,run_here,sites_Mn,vector_indices_Mn,LaMnO3)

## Plot van Vleck data

In [ ]:
fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})

alpha = 1
pointer_size = 5

ax.plot(
    np.array(data_dictionary["VanVleck_arctan_automated"]), 
    np.array(data_dictionary["VanVleck_mag_automated"]),
    color='tab:orange',
    marker='.',
    linewidth=0,
    label="automated",
    alpha=alpha,
    markersize=pointer_size
)
ax.plot(
    np.array(data_dictionary["VanVleck_arctan_specified"]), 
    np.array(data_dictionary["VanVleck_mag_specified"]),
    'g.',
    marker='.',
    linewidth=0,
    label="specified",
    alpha=alpha,
    markersize=pointer_size
)
ax.plot(
    np.array(data_dictionary["VanVleck_arctan_ignore"]), 
    np.array(data_dictionary["VanVleck_mag_ignore"]),
    'c.',
    label="ignore",
    alpha=alpha,
    markersize=pointer_size
)

# ax.set_rmax(0.7)
# ax.set_rmin(4.51)
# ax.set_rticks([0.49,0.495,0.5, 0.505, 0.51, 0.515])  
ax.set_rlabel_position(225)  
ax.set_thetagrids([0,60,120,180,240,300]) 
ax.grid(True)
plt.legend(loc='lower right')
plt.savefig("PolarPlot.svg")

In [ ]:
fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})

alpha = 1
pointer_size = 5

ax.plot(
    np.array(data_dictionary["VanVleck_arctan_automated"]), 
    np.array(data_dictionary["VanVleck_mag_automated"]),
    color='tab:orange',
    marker='.',
    linewidth=0,
    label="automated",
    alpha=alpha,
    markersize=pointer_size
)
# ax.plot(
#     np.array(data_dictionary["VanVleck_arctan_specified"]), 
#     np.array(data_dictionary["VanVleck_mag_specified"]),
#     'g.',
#     marker='.',
#     linewidth=0,
#     label="specified",
#     alpha=alpha,
#     markersize=pointer_size
# )
# ax.plot(
#     np.array(data_dictionary["VanVleck_arctan_ignore"]), 
#     np.array(data_dictionary["VanVleck_mag_ignore"]),
#     'c.',
#     label="ignore",
#     alpha=alpha,
#     markersize=pointer_size
# )

# ax.set_rmax(0.7)
# ax.set_rmin(4.51)
# ax.set_rticks([0.49,0.495,0.5, 0.505, 0.51, 0.515])  
ax.set_rlabel_position(225)  
ax.set_thetagrids([0,60,120,180,240,300]) 
ax.grid(True)
plt.legend(loc='lower right')
plt.savefig("PolarPlot_automated.svg")

In [ ]:
fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})

alpha = 1
pointer_size = 5

# ax.plot(
#     np.array(data_dictionary["VanVleck_arctan_automated"]), 
#     np.array(data_dictionary["VanVleck_mag_automated"]),
#     color='tab:orange',
#     marker='.',
#     linewidth=0,
#     label="automated",
#     alpha=alpha,
#     markersize=pointer_size
# )
ax.plot(
    np.array(data_dictionary["VanVleck_arctan_specified"]), 
    np.array(data_dictionary["VanVleck_mag_specified"]),
    'g.',
    marker='.',
    linewidth=0,
    label="specified",
    alpha=alpha,
    markersize=pointer_size
)
# ax.plot(
#     np.array(data_dictionary["VanVleck_arctan_ignore"]), 
#     np.array(data_dictionary["VanVleck_mag_ignore"]),
#     'c.',
#     label="ignore",
#     alpha=alpha,
#     markersize=pointer_size
# )

# ax.set_rmax(0.7)
# ax.set_rmin(4.51)
# ax.set_rticks([0.49,0.495,0.5, 0.505, 0.51, 0.515])  
ax.set_rlabel_position(225)  
ax.set_thetagrids([0,60,120,180,240,300]) 
ax.grid(True)
plt.legend(loc='lower right')
plt.savefig("PolarPlot_specified.svg")

In [ ]:
fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})

alpha = 1
pointer_size = 5

# ax.plot(
#     np.array(data_dictionary["VanVleck_arctan_automated"]), 
#     np.array(data_dictionary["VanVleck_mag_automated"]),
#     'g.',
#     marker='.',
#     linewidth=0,
#     label="automated",
#     alpha=alpha,
#     markersize=pointer_size
# )
# ax.plot(
#     np.array(data_dictionary["VanVleck_arctan_specified"]), 
#     np.array(data_dictionary["VanVleck_mag_specified"]),
#     color='tab:orange',
#     marker='.',
#     linewidth=0,
#     label="specified",
#     alpha=alpha,
#     markersize=pointer_size
# )
ax.plot(
    np.array(data_dictionary["VanVleck_arctan_ignore"]), 
    np.array(data_dictionary["VanVleck_mag_ignore"]),
    'c.',
    label="ignore",
    alpha=alpha,
    markersize=pointer_size
)

# ax.set_rmax(0.7)
# ax.set_rmin(4.51)
# ax.set_rticks([0.49,0.495,0.5, 0.505, 0.51, 0.515])  
ax.set_rlabel_position(225)  
ax.set_thetagrids([0,60,120,180,240,300]) 

ax.grid(True)
plt.legend(loc='lower right')
plt.savefig("PolarPlot_ignore.svg")

## Plot shortest to longest bond lengths

In [ ]:
smallest_MnO6_1 = []
smallest_MnO6_2 = []
smallest_MnO6_3 = []
smallest_MnO6_4 = []
smallest_MnO6_5 = []
smallest_MnO6_6 = []

for octahedron in MnO6_octahedra:
    
    vertex_bond_lengths = sorted(octahedron.core_ligand_bond_lengths)
    
    smallest_MnO6_1.append(vertex_bond_lengths[0])
    smallest_MnO6_2.append(vertex_bond_lengths[1])
    smallest_MnO6_3.append(vertex_bond_lengths[2])
    smallest_MnO6_4.append(vertex_bond_lengths[3])
    smallest_MnO6_5.append(vertex_bond_lengths[4])
    smallest_MnO6_6.append(vertex_bond_lengths[5])

MnO6_bonds = [
    smallest_MnO6_1,
    smallest_MnO6_2,
    smallest_MnO6_3,
    smallest_MnO6_4,
    smallest_MnO6_5,
    smallest_MnO6_6,
]

In [ ]:
suffix=["st ","nd","rd","th","th","th"]

plt.figure()
for i in range(6):
    plt.hist(MnO6_bonds[i], bins=np.linspace(1.7,2.6,100).tolist(), alpha=0.5,label="{}{} bond length".format(i+1,suffix[i]))
plt.legend()
plt.xlabel("Length (Å)")
plt.ylabel("Number of Mn-O bonds")
for i in [1.9055,1.9683,2.1800]:
    plt.axvline(i)
plt.savefig("Mn-O_bond-lengths.svg")

## $Q_1$, $Q_4$ - $Q_6$ modes

In [ ]:
for i in ["1","4","5","6"]:
    
    ignore_data = data_dictionary["VanVleck_Q{}_ignore".format(i)]
    ignore_data = [i for i in ignore_data if type(i)!=type(None)]
    av_ignore = np.average(ignore_data)
    err_ignore = np.std(ignore_data)
    
    automated_data = data_dictionary["VanVleck_Q{}_automated".format(i)]
    automated_data = [i for i in automated_data if type(i)!=type(None)]
    av_automated = np.average(automated_data)
    err_automated = np.std(automated_data)
    
    specified_data = data_dictionary["VanVleck_Q{}_specified".format(i)]
    specified_data = [i for i in specified_data if type(i)!=type(None)]
    av_specified = np.average(specified_data)
    err_specified = np.std(specified_data)
    
    print("For ignore_data, {} value is {}±{}.\n".format(i,av_ignore,err_ignore))
    print("For automated_data, {} value is {}±{}.\n".format(i,av_automated,err_automated))
    print("For specified_data, {} value is {}±{}.\n\n".format(i,av_specified,err_specified))

In [ ]:
plt.figure()
for i in ["4","5","6"]:
    plt.scatter(
        data_dictionary["bond_angle_variance"],
        data_dictionary["VanVleck_Q{}_automated".format(i)],
        label="Q{}".format(i)
    )
plt.xlabel("Bond angle variance ($^{\circ2}$)")
plt.ylabel("Octahedral shear distortion modes")
plt.legend()

In [ ]:
plt.figure()
plt.scatter(data_dictionary["bond_angle_variance"],data_dictionary["VanVleck_Q1_automated"])
plt.xlabel("Bond angle variance ($^{\circ2}$)")
plt.ylabel("$Q_1$ distortion mode (Å)")

In [ ]:
plt.figure()
plt.hist(
    [i for i in data_dictionary["VanVleck_Q4_automated"] if type(i)!=type(None)], 
    bins=np.linspace(-1,1,100).tolist(), alpha=0.5,label="$Q_4$"
)
plt.hist(
    [i for i in data_dictionary["VanVleck_Q5_automated"] if type(i)!=type(None)], 
    bins=np.linspace(-1,1,100).tolist(), alpha=0.5,label="$Q_5$"
)
plt.hist(
    [i for i in data_dictionary["VanVleck_Q6_automated"] if type(i)!=type(None)], 
    bins=np.linspace(-1,1,100).tolist(), alpha=0.5,label="$Q_6$"
)
plt.legend()
plt.xlabel("$Q$ magnitude (Å)")
plt.ylabel("Number of octahedra")
plt.savefig("shear-modes.svg",transparent=True,bbox_inches="tight")

## $Q_2$ / $Q_3$ heat maps

In [ ]:
hist_data = np.array([
    i for i in
    np.array([data_dictionary["VanVleck_Q3_1_automated"],data_dictionary["VanVleck_Q2_automated"]]).T 
    if (type(i[0]) != type(None) and type(i[1]) != type(None))
]).T

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=1)

axes.set_title('LaMnO$_3$: $E_g(Q_2,Q_3)$ plot')
interval = 0.015
hist_range = np.arange(-0.5, 0.5+interval, interval)
axes.hist2d(hist_data[0], hist_data[1],bins=(hist_range, hist_range))
axes.set_xlabel("$Q_3$ (Å)")
axes.set_ylabel("$Q_2$ (Å)")
axes.set_ylim(-0.5,0.5)
axes.set_xlim(-0.5,0.5)